In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from copy import deepcopy
import numpy as np
import pytreenet as ptn
import random
from numpy.random import default_rng, Generator
from fractions import Fraction

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Tree Tensor Network Operator for the Toy Model
In the main manuscript we consider a toy Hamiltonian to show the workings of our algorithm to find tree tensor network operators from a Hamiltonian. The toy Hamiltonian is defined on a tree structure as given in Fig.4.1a) in the main manuscript as
$$
H_{\text{toy}}= \sum_{j=1}^4 h_j = Y_2 X_3 X_4 + X_1 Y_2 Y_6 + X_1 Y_2 Z_5 + Z_5 X_7 X_8.
$$
Here we want to show that our implemented algorithm yields the same state diagrams as the ones shown in the main text.

In [3]:
# Building the tree
def construct_reference_tree() -> ptn.TreeTensorNetworkState:
    """
    Generates the desired tree tensor network used as a reference to construct
     the Hamiltonian.
    """
    ttns = ptn.TreeTensorNetworkState()
    # Physical legs come last
    node1, tensor1 = ptn.random_tensor_node((1, 1, 2), identifier="site1")
    node2, tensor2 = ptn.random_tensor_node((1, 1, 1, 2), identifier="site2")
    node3, tensor3 = ptn.random_tensor_node((1, 2), identifier="site3")
    node4, tensor4 = ptn.random_tensor_node((1, 2), identifier="site4")
    node5, tensor5 = ptn.random_tensor_node((1, 1, 1, 2), identifier="site5")
    node6, tensor6 = ptn.random_tensor_node((1, 2), identifier="site6")
    node7, tensor7 = ptn.random_tensor_node((1, 1, 2), identifier="site7")
    node8, tensor8 = ptn.random_tensor_node((1, 2), identifier="site8")
    
    ttns.add_root(node1, tensor1)
    ttns.add_child_to_parent(node2, tensor2, 0, "site1", 0)
    ttns.add_child_to_parent(node3, tensor3, 0, "site2", 1)
    ttns.add_child_to_parent(node4, tensor4, 0, "site2", 2)
    ttns.add_child_to_parent(node5, tensor5, 0, "site1", 1)
    ttns.add_child_to_parent(node6, tensor6, 0, "site5", 1)
    ttns.add_child_to_parent(node7, tensor7, 0, "site5", 2)
    ttns.add_child_to_parent(node8, tensor8, 0, "site7", 1)
    return ttns

In [4]:
def construct_toy_hamiltonian() -> ptn.Hamiltonian:
    paulis = ptn.pauli_matrices()
    conversion_dict = {"X": paulis[0], "Y": paulis[1], "Z": paulis[2], "I2": np.eye(2)}
    
    ham = [{'site8': 'I2', 'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site4': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site8': 'Z', 'site1': 'X', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'X', 'site4': 'Z', 'site6': 'Y', 'site7': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site8': 'I2'}]
    #coeffs = [1, 1, 1, 1]
    
    return ptn.Hamiltonian([ptn.TensorProduct(i) for i in ham], conversion_dictionary=conversion_dict) #, coeffs = [random.randint(1, 100) for _ in range(18)])



In [5]:
def construct_hamiltonian(ham, coeffs_mapping, coeffs= None, ) -> ptn.Hamiltonian:

    if coeffs is None:
        coeffs = [1 for _ in range(len(ham))]
    
    paulis = ptn.pauli_matrices()
    conversion_dict = {"X": paulis[0], "Y": paulis[1], "Z": paulis[2], "I2": np.eye(2)}
    return ptn.Hamiltonian([ptn.TensorProduct(i) for i in ham], conversion_dictionary=conversion_dict, coeffs=coeffs, coeffs_mapping=coeffs_mapping)


In [6]:
def construct_toy_hamiltonian() -> ptn.Hamiltonian:
    paulis = ptn.pauli_matrices()
    conversion_dict = {"X": paulis[0], "Y": paulis[1], "Z": paulis[2], "I2": np.eye(2)}
    term1 = ptn.TensorProduct({"site2": "Y", "site3": "X", "site4": "X"})
    term2 = ptn.TensorProduct({"site1": "X", "site2": "Y", "site6": "Y"})
    term3 = ptn.TensorProduct({"site1": "X", "site2": "Y", "site5": "Z"})
    term4 = ptn.TensorProduct({"site5": "Z", "site7": "X", "site3": "X"})
    terms = [term1, term2, term3, term4]
    coeffs = [(Fraction(2,3),"2"), (Fraction(1,4),"3"), (Fraction(1,3),"4"), (Fraction(1,2),"5")]
    ham_terms =  [(x, y, z) for (x, y), z in zip(coeffs, terms)]
    coeffs_mapping = {"1":1, "2":2, "3":3, "4":4, "5":5}
    
    return ptn.Hamiltonian(ham_terms, conversion_dictionary=conversion_dict ,coeffs_mapping=coeffs_mapping)

In [7]:

#hamiltonian_toy = construct_hamiltonian(ham, ham_dict, coeffs)
hamiltonian_toy = construct_toy_hamiltonian()
ttns_root1 = construct_reference_tree()
hamiltonian1 = hamiltonian_toy.pad_with_identities(ttns_root1)
#hamiltonian1 = hamiltonian_toy

### The Single Term Diagrams

In [8]:
for i, term in enumerate(hamiltonian1.terms):
    single_term_diag = ptn.SingleTermDiagram.from_single_term(term, ttns_root1)
    print(f"State Diagram for term {i}")
    print(single_term_diag)
    print("-----------------")

State Diagram for term 0
Hyperedges: {'site1': 'I2 (2/3 * 2)', 'site2': 'Y (1 * 1)', 'site3': 'X (1 * 1)', 'site4': 'X (1 * 1)', 'site5': 'I2 (1 * 1)', 'site6': 'I2 (1 * 1)', 'site7': 'I2 (1 * 1)', 'site8': 'I2 (1 * 1)'}
Vertices: [('site1', 'site2'), ('site1', 'site5'), ('site2', 'site3'), ('site2', 'site4'), ('site5', 'site6'), ('site5', 'site7'), ('site7', 'site8')]
-----------------
State Diagram for term 1
Hyperedges: {'site1': 'X (1/4 * 3)', 'site2': 'Y (1 * 1)', 'site3': 'I2 (1 * 1)', 'site4': 'I2 (1 * 1)', 'site5': 'I2 (1 * 1)', 'site6': 'Y (1 * 1)', 'site7': 'I2 (1 * 1)', 'site8': 'I2 (1 * 1)'}
Vertices: [('site1', 'site2'), ('site1', 'site5'), ('site2', 'site3'), ('site2', 'site4'), ('site5', 'site6'), ('site5', 'site7'), ('site7', 'site8')]
-----------------
State Diagram for term 2
Hyperedges: {'site1': 'X (1/3 * 4)', 'site2': 'Y (1 * 1)', 'site3': 'I2 (1 * 1)', 'site4': 'I2 (1 * 1)', 'site5': 'Z (1 * 1)', 'site6': 'I2 (1 * 1)', 'site7': 'I2 (1 * 1)', 'site8': 'I2 (1 * 1)'}

We can see that the single term diagrams are exactly the one we depicted in Fig. 4.1b).

### Complete State Diagram

In [9]:
state_diagram1 = ptn.StateDiagram.from_hamiltonian(hamiltonian1, ttns_root1, ptn.state_diagram.TTNOFinder.SGE)
print(len(state_diagram1.get_all_vertices()))
print(state_diagram1)
#ptn.TTNO.from_hamiltonian(hamiltonian1, ttns_root1)
#print(state_diagram1.coeffs, state_diagram1.coeffs_indices)
print(hamiltonian1.coeffs_mapping)

15
hyperedges:
label = I2; corr_site = site1; coeff = 2/3 * 2; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = X; corr_site = site1; coeff = 1 * 1; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = I2; corr_site = site1; coeff = 1/2 * 5; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = Y; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = Y; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = I2; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = X; corr_site = site3; coeff = 1 * 1; connected to ('site2', 'site3'), 
label = I2; corr_site = site3; coeff = 1 * 1; connected to ('site2', 'site3'), 
label = X; corr_site = site4; coeff = 1 * 1; connected to ('site2', 'site4'), 
label = I2; corr_site = site4; coeff = 1 * 1; connected to ('site2', 'site4'), 
label 

In [10]:
state_diagram2 = ptn.StateDiagram.from_hamiltonian(hamiltonian1, ttns_root1, ptn.TTNOFinder.TREE)
print(len(state_diagram2.get_all_vertices()))
print(state_diagram2)
#ptn.TTNO.from_hamiltonian(hamiltonian1, ttns_root1)


15
hyperedges:
label = I2; corr_site = site1; coeff = 2/3 * 2; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = X; corr_site = site1; coeff = 1/4 * 3; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = I2; corr_site = site1; coeff = 1/2 * 5; connected to ('site1', 'site2'), ('site1', 'site5'), 
label = Y; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = Y; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = I2; corr_site = site2; coeff = 1 * 1; connected to ('site1', 'site2'), ('site2', 'site3'), ('site2', 'site4'), 
label = X; corr_site = site3; coeff = 1 * 1; connected to ('site2', 'site3'), 
label = I2; corr_site = site3; coeff = 1 * 1; connected to ('site2', 'site3'), 
label = X; corr_site = site4; coeff = 1 * 1; connected to ('site2', 'site4'), 
label = I2; corr_site = site4; coeff = 1 * 1; connected to ('site2', 'site4'), 
labe

### Compare tensor values

In [11]:
def _find_permutation_rec(ttno, leg_dict, node_id, perm):
    node, _ = ttno[node_id]
    input_index = leg_dict[node_id]
    output_index = input_index + len(ttno.nodes)
    perm.extend([output_index, input_index ])
    if not node.is_leaf():
        for child_id in node.children:
            _find_permutation_rec(ttno, leg_dict, child_id, perm)

In [15]:
leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4,
                "site6": 5, "site7": 6, "site8": 7}

original_tensor = hamiltonian1.to_tensor(ttns_root1).operator#.transpose([3,0,4,1,5,2])

ttno = ptn.TTNO.from_hamiltonian(hamiltonian1, ttns_root1, ptn.state_diagram.TTNOFinder.BASE)


#print(ttno.tensors)

contructed_tensor = ttno.completely_contract_tree(to_copy=True)[0]

permutation = []
_find_permutation_rec(ttno, leg_dict, ttno.root_id, permutation)
correct_tensor = original_tensor.transpose(permutation)
#print(permutation)


print("Equality: ", np.allclose(correct_tensor,contructed_tensor))
#print(original_tensor)
print("---------------------------")
#print(contructed_tensor)

Equality:  True
---------------------------


One can see that this state diagram corresponds to the state diagram given in Fig. 4.1c) in the main text.

In [11]:
def obtain_bond_dimensions(ttno: ptn.TTNO) -> np.ndarray:
    """
    Obtains the bond dimensions of a TTN.

    Args:
        ttno (ptn.TTNO): The TTN for which to determine the bond dimensions.

    Returns:
        np.ndarray: A 1D-array containing all bond-dimensions
    """
    dimensions = []
    for node_id in ttno.nodes:
        node = ttno.nodes[node_id]
        if not node.is_root():
            dimensions.append(node.parent_leg_dim())
    return np.asarray(dimensions)

In [12]:
leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4,
                "site6": 5, "site7": 6, "site8": 7,"site9": 8}
ttno_ham = ptn.TTNO.from_hamiltonian(hamiltonian1, ttns_root1)
total_tensor = hamiltonian1.to_tensor(ttns_root1).operator
ttno_svd = ptn.TTNO.from_tensor(ttns_root1,
                                total_tensor,
                                leg_dict,
                                mode=ptn.Decomposition.tSVD)
ham_dim = obtain_bond_dimensions(ttno_ham)
svd_dim = obtain_bond_dimensions(ttno_svd)

print("ham_dim",ham_dim,"svd_dim", svd_dim)
if np.any(ham_dim > svd_dim):
    print(ham_dim, svd_dim)

ham_dim [1 1 1 1 1 1 1] svd_dim [1 1 1 1 1 1 1]


## Find Minimum problematic hamiltonian

In [ ]:
ham = [{'site4': 'Z', 'site2': 'Y', 'site8': 'Y', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'Y', 'site1': 'Z', 'site3': 'X', 'site2': 'Z', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site7': 'Z', 'site8': 'Y', 'site2': 'I2', 'site6': 'X', 'site5': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2'}, {'site3': 'I2', 'site4': 'Z', 'site7': 'X', 'site8': 'Z', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site6': 'I2', 'site5': 'Y', 'site1': 'X', 'site8': 'X', 'site4': 'Z', 'site7': 'X', 'site2': 'I2', 'site3': 'I2'}, {'site6': 'I2', 'site1': 'Y', 'site5': 'Z', 'site4': 'I2', 'site2': 'I2', 'site3': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'I2', 'site4': 'X', 'site2': 'X', 'site1': 'X', 'site6': 'I2', 'site5': 'Z', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'X', 'site2': 'Z', 'site1': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site7': 'I2', 'site2': 'Z', 'site4': 'X', 'site3': 'Y', 'site8': 'I2', 'site1': 'I2', 'site6': 'I2'}, {'site7': 'X', 'site4': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Z', 'site7': 'X', 'site2': 'I2', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site2': 'I2', 'site6': 'Y', 'site8': 'Z', 'site1': 'Z', 'site3': 'Z', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site2': 'I2', 'site3': 'I2', 'site6': 'Z', 'site7': 'Z', 'site4': 'Z', 'site8': 'Z', 'site1': 'I2', 'site5': 'I2'}, {'site8': 'Y', 'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2'}, {'site7': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site8': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}]
coeffs = [-4.67, -2.9633104276996303, 6.680706729782859, -4.30560120789554, 5.202483584181863, 8.505121153194981, -4.011735853871354, -7.643281057886108, 6.157174317449904, 9.561269374270271, 2.9385071692017757, -8.54746490019713, 7.798682301513789, -4.6568899743444785, 9.241682837178718, -6.285495010325322, -5.907195938916027, -4.12665854873854]
leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4, "site6": 5, "site7": 6, "site8": 7,"site9": 8}
        
for i in range(len(ham)):
    c = coeffs[i]
    coeffs[i] = coeffs[i]+10
    hamiltonian_temp = construct_hamiltonian(ham,coeffs)
    ttns_root = construct_reference_tree()
    ham_t = hamiltonian_temp.pad_with_identities(ttns_root)

    ttno_ham = ptn.TTNO.from_hamiltonian(ham_t, ttns_root)
    total_tensor = ham_t.to_tensor(ttns_root).operator
    ttno_svd = ptn.TTNO.from_tensor(ttns_root,
                                    total_tensor,
                                    leg_dict,
                                    mode=ptn.Decomposition.tSVD)
    ham_dim = obtain_bond_dimensions(ttno_ham)
    svd_dim = obtain_bond_dimensions(ttno_svd)


    

    if np.any(ham_dim > svd_dim):
        print(i, ham_dim, svd_dim, "still broken for")
    coeffs[i] = c
        

In [ ]:

ham, coeffs = [{'site1': 'X', 'site8': 'Y', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'Y', 'site8': 'I2', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'I2', 'site7': 'Z', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site6': 'Z', 'site1': 'I2', 'site7': 'I2', 'site5': 'Z', 'site8': 'Y', 'site2': 'X', 'site3': 'I2', 'site4': 'I2'}, {'site6': 'Z', 'site1': 'I2', 'site8': 'X', 'site3': 'Z', 'site2': 'X', 'site7': 'X', 'site4': 'I2', 'site5': 'I2'}, {'site7': 'Z', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site8': 'I2'}, {'site3': 'Z', 'site5': 'X', 'site6': 'X', 'site2': 'Z', 'site8': 'I2', 'site1': 'I2', 'site4': 'I2', 'site7': 'I2'}, {'site4': 'Z', 'site8': 'X', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site3': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Z', 'site7': 'Z', 'site6': 'Y', 'site3': 'I2', 'site1': 'I2', 'site8': 'X', 'site2': 'I2', 'site5': 'I2'}, {'site5': 'Z', 'site8': 'I2', 'site2': 'I2', 'site4': 'I2', 'site1': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'Z', 'site3': 'Z', 'site2': 'I2', 'site1': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'X', 'site1': 'X', 'site8': 'Y', 'site4': 'X', 'site2': 'X', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site8': 'X', 'site5': 'I2', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2'}], [(Fraction(6, 1), '9'), (Fraction(3, 10), '12'), (Fraction(3, 1), '5'), (Fraction(3, 4), '2'), (Fraction(6, 7), '12'), (Fraction(-8, 1), '12'), (Fraction(-9, 10), '4'), (Fraction(1, 4), '10'), (Fraction(1, 1), '5'), (Fraction(1, 1), '4'), (Fraction(4, 9), '5'), (Fraction(-1, 1), '2'), (Fraction(4, 9), '14'), (Fraction(-4, 5), '3')]
ham_dict = {'2': 2.835697909530385, '3': 4.563938233861004, '4': 9.061058628831027, '5': 8.32944716069347, '6': 6.684243698012364, '7': 5.819053198073945, '8': 8.726491608447832, '9': 6.686432680473677, '10': 7.0935373289106955, '11': 4.444348450120315, '12': 7.854479877498686, '13': 9.703525270953671, '14': 9.802210710023699, '15': 9.712784232553258}
leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4, "site6": 5, "site7": 6, "site8": 7,"site9": 8}
        

term_eliminated = 0

while True:
    error = False
    for i in range(len(ham)):
        t = ham.pop(i)
        c = coeffs.pop(i)
        
        hamiltonian_temp = construct_hamiltonian(ham,ham_dict,coeffs)
        ttns_root = construct_reference_tree()
        ham_t = hamiltonian_temp.pad_with_identities(ttns_root)

        ttno_ham = ptn.TTNO.from_hamiltonian(ham_t, ttns_root)
        total_tensor = ham_t.to_tensor(ttns_root).operator
        ttno_svd = ptn.TTNO.from_tensor(ttns_root,
                                        total_tensor,
                                        leg_dict,
                                        mode=ptn.Decomposition.tSVD)
        ham_dim = obtain_bond_dimensions(ttno_ham)
        svd_dim = obtain_bond_dimensions(ttno_svd)


        

        if np.any(ham_dim > svd_dim):
            error = True
            term_eliminated += 1
            break
        else:
            ham.insert(i, t)
            coeffs.insert(i, c)
        
    if not error :
        break

print(term_eliminated, " terms eliminated") 
print(ham," , ",coeffs)

In [ ]:
ham, coeffs = [{'site6': 'I2', 'site1': 'I2', 'site2': 'Y', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site5': 'I2', 'site8': 'I2', 'site2': 'Y', 'site1': 'X', 'site6': 'I2', 'site4': 'I2', 'site3': 'I2', 'site7': 'I2'}, {'site5': 'I2', 'site3': 'Z', 'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'Y', 'site5': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site1': 'X', 'site8': 'X', 'site4': 'Z', 'site7': 'X', 'site3': 'I2', 'site5': 'Z', 'site2': 'I2', 'site6': 'I2'}, {'site3': 'Z', 'site6': 'Z', 'site5': 'Z', 'site1': 'I2', 'site2': 'I2', 'site4': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site2': 'I2', 'site3': 'Y', 'site1': 'Y', 'site4': 'X', 'site8': 'X', 'site7': 'X', 'site5': 'I2'}, {'site8': 'X', 'site7': 'Z', 'site4': 'Z', 'site2': 'I2', 'site1': 'X', 'site5': 'X', 'site3': 'I2', 'site6': 'I2'}, {'site7': 'I2', 'site8': 'I2', 'site2': 'X', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2'}, {'site8': 'X', 'site2': 'Y', 'site1': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site6': 'I2', 'site7': 'I2'}, {'site5': 'Y', 'site7': 'Z', 'site2': 'I2', 'site8': 'Y', 'site4': 'Y', 'site1': 'Y', 'site3': 'X', 'site6': 'I2'}, {'site2': 'Y', 'site3': 'I2', 'site8': 'Z', 'site7': 'Y', 'site5': 'X', 'site1': 'I2', 'site4': 'I2', 'site6': 'I2'}, {'site5': 'Y', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site6': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site4': 'I2', 'site6': 'Y', 'site3': 'I2', 'site1': 'I2', 'site2': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site3': 'Z', 'site4': 'I2', 'site6': 'X', 'site2': 'Z', 'site1': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Y', 'site8': 'I2', 'site1': 'Y', 'site6': 'I2', 'site5': 'I2', 'site3': 'I2', 'site4': 'I2', 'site7': 'I2'}, {'site1': 'Z', 'site2': 'Z', 'site4': 'Y', 'site8': 'X', 'site7': 'I2', 'site5': 'X', 'site6': 'X', 'site3': 'I2'}, {'site3': 'Z', 'site6': 'Y', 'site7': 'X', 'site8': 'I2', 'site1': 'Z', 'site2': 'I2', 'site4': 'I2', 'site5': 'I2'}, {'site6': 'X', 'site1': 'I2', 'site2': 'I2', 'site3': 'I2', 'site4': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site6': 'I2', 'site2': 'I2', 'site7': 'Y', 'site8': 'X', 'site3': 'X', 'site4': 'Z', 'site1': 'I2', 'site5': 'I2'}, {'site2': 'I2', 'site5': 'Z', 'site1': 'Y', 'site7': 'I2', 'site8': 'Y', 'site3': 'Z', 'site4': 'I2', 'site6': 'I2'}, {'site4': 'Y', 'site1': 'Z', 'site6': 'I2', 'site2': 'I2', 'site3': 'I2', 'site5': 'I2', 'site7': 'I2', 'site8': 'I2'}, {'site2': 'Z', 'site8': 'X', 'site3': 'Y', 'site1': 'Y', 'site6': 'Y', 'site5': 'X', 'site4': 'I2', 'site7': 'I2'}], [-3, 1, 3, 3, -1, 5, -3, -4, 1, -3, -3, -5, -3, -4, 5, 2, -3, 1, 2, -1, -4, 5, -3]

leg_dict = {"site1": 0, "site2": 1, "site3": 2, "site4": 3, "site5": 4, "site6": 5, "site7": 6, "site8": 7,"site9": 8}
        

term_eliminated = 0
while True:
    error = False
    for i in range(len(ham)):
        t = ham.pop(i)
        c = coeffs.pop(i)
        hamiltonian_temp = construct_hamiltonian(ham, coeffs)
        ttns_root = construct_reference_tree()
        ham_t = hamiltonian_temp.pad_with_identities(ttns_root)

        ttno_ham = ptn.TTNO.from_hamiltonian(ham_t, ttns_root)
        
        total_tensor = ham_t.to_tensor(ttns_root).operator
        ttno_svd = ptn.TTNO.from_tensor(ttns_root,
                                        total_tensor,
                                        leg_dict,
                                        mode=ptn.Decomposition.tSVD)

        contructed_tensor = ttno_ham.completely_contract_tree(to_copy=True)[0]

        permutation = []
        _find_permutation_rec(ttno_ham, leg_dict, ttno_ham.root_id, permutation)
        #print(permutation)
        correct_tensor = total_tensor.transpose(permutation)

        if not np.allclose(correct_tensor,contructed_tensor):
            error = True
            term_eliminated += 1
            break
        else:
            ham.insert(i, t)
            coeffs.insert(i, c)
        
    if not error :
        break

print(term_eliminated, " terms eliminated") 
print(ham, coeffs)